In [ ]:
# Parameters
CONTEXTLENGTH = 512  # will be overridden by papermill

this notebook loads timesfm and makes predictions using it. Input context data is loaded from data/interim/data.csv. results are saved to data/processed/prediction_matrices/50/.

In [1]:
import pandas as pd
import torch
import numpy as np
import logging
from pyprojroot import here

import sys
sys.path.append(str(here() / 'scripts'))
#sys.path.append(str(here() / 'timesfm'))

from transformers import AutoModelForCausalLM
from utils import (get_data,get_y_context, save)

/home/valtti/code/ed-ml-multivar/env-sundial/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [ ]:
MAXMODELCONTEXT= 2880 # max context length for this model
MAXCONTEXT = 12840 # max context length for this dataset (2017-01-01 - 2018-06-19)
if CONTEXTLENGTH > MAXMODELCONTEXT:
    CONTEXTLENGTH = MAXMODELCONTEXT

if CONTEXTLENGTH > MAXCONTEXT:
    CONTEXTLENGTH = MAXCONTEXT

MODEL = 'sundial'
TARGET = 'occ'
FEATURESET = 'u'
TESTNAME = f'{TARGET}-{MODEL}-{FEATURESET}-{CONTEXTLENGTH}'
TESTSTART = pd.Timestamp('20180620')
TESTSIZE = 365*24
PREDICTIONLENGTH = 24

In [ ]:
logging.basicConfig(
        format='%(asctime)s - %(message)s', 
        level=logging.INFO,
        filename=here() / f"logs/logger/{TESTNAME}.log",      
        filemode='w'
        )
logging.info('Starting test.')
logging.info(f'MODEL: {MODEL} TARGET: {TARGET} FEATURESET:'
     f'{FEATURESET} CONTEXTLENGTH: {CONTEXTLENGTH}')

In [ ]:
data = get_data(TARGET, MODEL)
cd = {
        'occ' : 'Target:Occupancy',
        'arr' : 'Target:Arrivals'
    }
target_name = cd[TARGET]
y = data[target_name]
y_context = get_y_context(data, TARGET, CONTEXTLENGTH, TESTSTART).astype(np.float32)

y_context = torch.from_numpy(y_context.to_numpy())

torch.float32

In [5]:
model = AutoModelForCausalLM.from_pretrained('thuml/sundial-base-128m', trust_remote_code=True)

/home/valtti/code/ed-ml-multivar/env-sundial/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/thuml/sundial-base-128m:
- configuration_sundial.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/thuml/sundial-base-128m:
- ts_generation_mixin.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/thuml/sundial-base-128m:
- flow_loss.py
.

In [ ]:
num_samples = 100
forecast = model.generate(y_context, max_new_tokens=PREDICTIONLENGTH, num_samples=num_samples) # (batch_size, samples, horizon)

In [25]:
y_mean = torch.mean(forecast, dim=1)
save(model_name=MODEL, featureset_name=FEATURESET, target_name=TARGET, context_length=CONTEXTLENGTH,
    test_start=TESTSTART, y_pred=y_mean)